In [ ]:
import pandas as pd
import numpy as np
import re

FILE1 = 'salix008_tab.txt'
FILE2 = "Wheatfen_treecores.xlsx"
TREE = "Alnus"

with open(FILE1,'r') as f:
    out = f.read()
    out = re.sub("(  +)", ",", out)
    out = re.sub("\*\*\*\*\*\*\*", "", out)
    with open(f"{FILE}_processed",'w') as o:
        o.write(out)

salix_df = pd.read_csv(f"{FILE1}_processed")
wheatfen_df = pd.read_excel(FILE2)

wheatfen_df_diameter = wheatfen_df[['ID', 'Diameter/cm']].dropna()
wheatfen_df_diameter['ID'] = wheatfen_df_diameter['ID'].apply(lambda x : x.replace("?",""))

salix_df = salix_df.set_index(['year']).unstack().reset_index(name='ring_width')
salix_df = salix_df.rename(columns={'level_0':'ID'}).dropna().sort_values('year')

salix_df['ring_width_adjusted'] = salix_df['ring_width'] * 2 / 10
salix_df['ring_width_adjusted_cumulative'] = salix_df.iloc[::-1].groupby('ID')['ring_width_adjusted'].cumsum()[::-1]
salix_df['ring_width_adjusted_cumulative'] = salix_df.groupby('ID')['ring_width_adjusted_cumulative'].shift(-1)

def calc_diameter(x):
    d = wheatfen_df_diameter[(wheatfen_df_diameter['ID']==x['ID'][:-1])]['Diameter/cm'].values
    if len(d) != 1:
        return None
    d = d[0]
    if x['year'] != 2022:
        return d-x['ring_width_adjusted_cumulative']
    else:
        return d

salix_df['diameter']=salix_df.apply(calc_diameter,axis=1)
salix_df = salix_df.drop(['ring_width_adjusted', 'ring_width_adjusted_cumulative'],axis=1)

biomass_params = {"Alnus": (0.0859, 2.3537), "Acer": (-2.7606, 2.5189),"Quercus": (-2.3223,2.4029)}

salix_df['basal_area'] = salix_df.apply(lambda x : (np.pi / 4) * x['diameter'] ** 2, axis=1)
salix_df['biomass'] = salix_df.apply(lambda x : np.real(biomass_params[TREE][0] * x['diameter'] ** biomass_params[TREE][1]), axis=1)
salix_df['carbon'] = salix_df['biomass'] * 0.47
salix_df['delta_basal_area'] = salix_df.groupby("ID")['basal_area'].diff()
salix_df['delta_biomass'] = salix_df.groupby("ID")['biomass'].diff()
salix_df['delta_carbon'] = salix_df.groupby("ID")['carbon'].diff()

salix_df = salix_df.dropna()
salix_df